In [7]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dataset = load_dataset("squad_v2")
# dataset['train'] = dataset['train'].select(range(32))
# dataset['validation'] = dataset['validation'].select(range(32))

def tokenize_function(examples):
    return tokenizer(examples['question'], examples['context'], padding = 'max_length', truncation = True)

dataset = dataset.map(tokenize_function, batched=True)
dataset.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
print(dataset)

train = torch.utils.data.DataLoader(dataset['train'], batch_size = 8, shuffle = True)
val = torch.utils.data.DataLoader(dataset['validation'], batch_size = 8)

for batch in train:
    print(batch)
    print(batch.keys())
    break

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11873
    })
})
{'input_ids': tensor([[  101,  2054, 11583,  ...,     0,     0,     0],
        [  101,  1037,  2744,  ...,     0,     0,     0],
        [  101,  2043,  2001,  ...,     0,     0,     0],
        ...,
        [  101,  2024,  1996,  ...,     0,     0,     0],
        [  101,  2054,  2003,  ...,     0,     0,     0],
        [  101,  2054,  2106,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tenso

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)
optimizer = torch.optim.AdamW(model.parameters())

print(model.__class__)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'transformers.models.bert.modeling_bert.BertForQuestionAnswering'>


In [ ]:
from tqdm.auto import tqdm
epochs = 3

for epoch in range(epochs):
    model.train()

    for i, batch in tqdm(enumerate(train), total = len(train)):
        optimizer.zero_grad()

        